In [1]:
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")
from dotenv import load_dotenv; load_dotenv()
%load_ext autoreload
%autoreload 2

In [2]:
import json
import yaml
import re

In [3]:
from rag.embed import EmbedChunks
from rag.generate import get_sources_and_context, QueryAgent, ComparisonAgent
from rag.prompts import DOCUMENT_QA_SYSTEM_PROMPT, DOCUMENT_QA_USER_PROMPT_TEMPLATE, FINAL_RESPONSE_SYSTEM_PROMPT_TEMPLATE
from rag.config import MAX_CONTEXT_LENGTHS

In [4]:
with open('../config.YAML') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
with open('../data/Constituciones.json') as f:
    constituciones = json.load(f)

In [6]:
embedder = EmbedChunks(config["embedding_model"])

In [7]:
get_sources_and_context(query="Derecho a la educación",
                        embedding_model=embedder.embedding_model,
                        num_chunks=3,
                        id_constitucion=1)

([32, 31, 33],
 ['Capítulo III: DE LOS DERECHOS Y DEBERES CONSTITUCIONALES, Artículo 19',
  'Capítulo III: DE LOS DERECHOS Y DEBERES CONSTITUCIONALES, Artículo 19',
  'Capítulo III: DE LOS DERECHOS Y DEBERES CONSTITUCIONALES, Artículo 19'],
 'Source 1:\néste y sus niveles superiores. El segundo nivel de transición es obligatorio, siendo requisito para el ingreso a la educación básica.\nLa educación básica y la educación media son obligatorias, debiendo el Estado financiar un sistema gratuito con tal objeto, destinado a asegurar el acceso a ellas de toda la población. En el caso de la educación media este sistema, en conformidad a la ley, se extenderá hasta cumplir los 21 años de edad.\nCorresponderá al Estado, asimismo, fomentar el desarrollo de la educación en todos sus niveles; estimular la investigación científica y tecnológica, la creación artística y la protección e incremento del patrimonio cultural de la Nación.\nEs deber de la comunidad contribuir al desarrollo y perfeccionamie

In [8]:
get_sources_and_context(query="Derecho a la educación",
                        embedding_model=embedder.embedding_model,
                        num_chunks=3,
                        id_constitucion=3)

([809, 807, 808],
 ['CAPÍTULO II: DERECHOS Y LIBERTADES FUNDAMENTALES, GARANTÍAS Y DEBERES CONSTITUCIONALES, Artículo 16.',
  'CAPÍTULO II: DERECHOS Y LIBERTADES FUNDAMENTALES, GARANTÍAS Y DEBERES CONSTITUCIONALES, Artículo 16.',
  'CAPÍTULO II: DERECHOS Y LIBERTADES FUNDAMENTALES, GARANTÍAS Y DEBERES CONSTITUCIONALES, Artículo 16.'],
 'Source 1:\nlibertad de enseñanza\na)\tLas personas tienen el derecho de abrir, organizar, mantener y desarrollar establecimientos educacionales, sin otra limitación que las impuestas por el orden público y la seguridad del país.\nb)\tLa enseñanza estatal y la reconocida oficialmente no podrán orientarse a propagar tendencia político partidista alguna.\nc)\tSe reconoce el derecho y el deber preferente de las familias de escoger la educación de sus hijos o pupilos, atendiendo a su interés superior.\nd)\tEl Estado respetará la autonomía de las instituciones de educación superior, de conformidad a la ley.\n24.\tEl derecho a la cultura.\na)\tEl Estado resgua

In [10]:
query = "Derecho a la salud"

constitucion1_agent = QueryAgent(embedding_model_name=config["embedding_model"],
                                llm=config["chat_model"],
                                temperature=config["temperature"],
                                max_context_length=MAX_CONTEXT_LENGTHS[config["chat_model"]],
                                system_content=DOCUMENT_QA_SYSTEM_PROMPT.render(language='spanish'),
                                assistant_content="",
                                constitucion_id=1)

first_response = constitucion1_agent(query=query,
                                    num_chunks=1,
                                    stream=False)['answer']

first_response

'El derecho a la salud está garantizado por el Estado, quien tiene la responsabilidad de promover, proteger y recuperar la salud de los individuos. Además, el Estado debe coordinar y controlar las acciones relacionadas con la salud [1]. Cada persona tiene el derecho de elegir el sistema de salud al que desea acogerse, ya sea público o privado [1].'

In [75]:

constitucion2_agent = QueryAgent(embedding_model_name=config["embedding_model"],
                                llm=config["chat_model"],
                                temperature=config["temperature"],
                                max_context_length=MAX_CONTEXT_LENGTHS[config["chat_model"]],
                                system_content=DOCUMENT_QA_SYSTEM_PROMPT,
                                assistant_content="",
                                constitucion_id=3)

second_response = constitucion2_agent(query=query,
                                    num_chunks=3,
                                    stream=False)

second_response['answer']

'El derecho a la salud está protegido por el Estado, que garantiza el acceso libre, universal, igualitario y oportuno a acciones de promoción, prevención, protección y recuperación de la salud [1]. El Estado tiene el deber preferente de garantizar la ejecución de estas acciones a través de instituciones estatales o privadas [1]. Además, el Estado debe crear, preservar y coordinar una red de establecimientos de salud de acuerdo con estándares básicos y uniformes de calidad [1].'

In [72]:
second_response['user_content']

"\nNow it's your turn. Below are several numbered sources of information:\n------\nSource 1:\nlibertad de enseñanza\na)\tLas personas tienen el derecho de abrir, organizar, mantener y desarrollar establecimientos educacionales, sin otra limitación que las impuestas por el orden público y la seguridad del país.\nb)\tLa enseñanza estatal y la reconocida oficialmente no podrán orientarse a propagar tendencia político partidista alguna.\nc)\tSe reconoce el derecho y el deber preferente de las familias de escoger la educación de sus hijos o pupilos, atendiendo a su interés superior.\nd)\tEl Estado respetará la autonomía de las instituciones de educación superior, de conformidad a la ley.\n24.\tEl derecho a la cultura.\na)\tEl Estado resguarda el derecho a participar en la vida cultural y científica. Protege la libertad creativa y su libre ejercicio, promueve el desarrollo y la divulgación del conocimiento, de las artes, las ciencias, la tecnología, el patrimonio cultural y asegura el acceso

In [52]:
first_response

'El derecho a la educación está garantizado en la Constitución. Según el artículo 10º de la Constitución (Source 2), la educación tiene como objetivo el pleno desarrollo de la persona en todas las etapas de su vida. Los padres tienen el derecho preferente y el deber de educar a sus hijos, y el Estado debe otorgar especial protección a este derecho. Además, el Estado tiene la obligación de promover la educación parvularia y financiar un sistema gratuito a partir del nivel medio menor para asegurar el acceso a la educación. La libertad de enseñanza también está protegida, permitiendo a las personas abrir, organizar y mantener establecimientos educacionales (Source 1).\n\nSource 1 también menciona que la educación básica y la educación media son obligatorias, y el Estado debe financiar un sistema gratuito para asegurar el acceso a ellas de toda la población. En el caso de la educación media, este sistema se extenderá hasta los 21 años de edad. Además, corresponde al Estado fomentar el des

In [82]:
FINAL_RESPONSE_SYSTEM_PROMPT_TEMPLATE.render()

'\nYou are a Constitutional Lawyer. You are asked to give a brief response about \nthe differences of two constitutions.\n\nThe first constitution is the current one, and the second one is a proposed one.\nAlways refer to the first constitution as "" and the second one as "".\n\nPlease respond in .'

In [84]:
final_agent = ComparisonAgent(embedding_model_name=config["embedding_model"],
                        llm=config["chat_model"],
                        temperature=config["temperature"],
                        max_context_length=MAX_CONTEXT_LENGTHS[config["chat_model"]],
                        system_content=FINAL_RESPONSE_SYSTEM_PROMPT_TEMPLATE.render(document1_title=constituciones[str(1)],
                                                                                    document2_title=constituciones[str(3)],
                                                                                    language='spanish'),                                      
                        assistant_content="")

In [89]:
final_agent(query=query,
            first_response=re.sub(r'\[\d+\]', '', first_response),
            second_response=re.sub(r'\[\d+\]', '', second_response['answer']),
            stream=False)['answer']

'La principal diferencia entre la Constitución Actual 1980 y la Propuesta Convención 2022 en relación al derecho a la salud radica en la forma en que se garantiza y se accede a este derecho.\n\nLa Constitución Actual 1980 establece que el derecho a la salud está garantizado por el Estado, quien tiene la responsabilidad de promover, proteger y recuperar la salud de los individuos. Además, se reconoce el derecho de cada persona a elegir el sistema de salud al que desea acogerse, ya sea estatal o privado.\n\nPor otro lado, la Propuesta Convención 2022 establece que el derecho a la salud está protegido por el Estado, el cual garantiza el acceso libre, universal, igualitario y oportuno a acciones de promoción, prevención, protección y recuperación de la salud. El Estado tiene el deber preferente de garantizar la ejecución de estas acciones a través de instituciones estatales o privadas. Además, se establece la obligación del Estado de crear, preservar y coordinar una red de establecimientos

In [40]:
final_agent(query=query,
            first_response=first_response,
            second_response=second_response,
            stream=False)['user_content']

'\nPlease explain the differences of two constitutions about this topic: ¿Qué derechos me otorga la Constitución?.\n\nThe first constitution says the following about the topic: La Constitución asegura a todas las personas el derecho a la vida y a la integridad física y psíquica [1]. También garantiza la igualdad ante la ley, sin establecer diferencias arbitrarias [1]. Además, establece el deber de respeto a Chile y a sus emblemas nacionales [3]..\nThe second constitution says the following about the topic: La Constitución otorga una serie de derechos fundamentales, garantías y deberes constitucionales [1]. Estos derechos incluyen el derecho a la salud, a la vivienda, al agua y al saneamiento, a la seguridad social y a la educación [1]. Estos derechos deben ser protegidos y su ejercicio solo puede ser limitado en situaciones de excepción, como guerra externa o interna, grave conmoción interior, emergencia y calamidad pública, cuando afecten gravemente el normal desenvolvimiento de las i

In [41]:
final_agent(query=query,
            first_response=first_response,
            second_response=second_response,
            stream=False)['system_content']

'\nYou are a Constitutional Lawyer. You are asked to give a brief response about \nthe differences of two constitutions.\n\nThe first constitution is the current one, and the second one is a proposed one.\nAlways refer to the first constitution as "Constitución Actual 1980" and the second one as "Anteproyecto Expertos 2023".\n\nPlease respond in the same language as the topic is written.'

In [18]:
DOCUMENT_QA_SYSTEM_PROMPT

'\nPlease provide an answer based solely on the provided sources. When referencing information from a source, cite the appropriate source(s) using their corresponding numbers. Every answer should include at least one source citation. Only cite a source when you are explicitly referencing it. If none of the sources are helpful, you should indicate that. For example:\nSource 1:\nThe sky is red in the evening and blue in the morning.\nSource 2:\nWater is wet when the sky is red.\nQuery: When is water wet?\nAnswer: Water will be wet when the sky is red [2], which occurs in the evening [1].\n'

In [23]:
constitucion1_agent(query=query,
                    num_chunks=3,
                    stream=False)['user_content']

"\nNow it's your turn. Below are several numbered sources of information:\n------\nSource 1:\nCapítulo III: DE LOS DERECHOS Y DEBERES CONSTITUCIONALES\nArtículo 19\n\nLa Constitución asegura a todas las personas:\n1º.- El derecho a la vida y a la integridad física y psíquica de la persona.\nLa ley protege la vida del que está por nacer.\nLa pena de muerte sólo podrá establecerse por delito contemplado en ley aprobada con quórum calificado.\nSe prohíbe la aplicación de todo apremio ilegítimo.\nEl desarrollo científico y tecnológico estará al servicio de las personas y se llevará a cabo con respeto a la vida y a la integridad física y psíquica. La ley regulará los requisitos, condiciones y restricciones para su utilización en las personas, debiendo resguardar especialmente la actividad cerebral, así como la información proveniente de ella;\n2º.- La igualdad ante la ley. En Chile no hay persona ni grupo privilegiados. En Chile no hay esclavos y el que pise su territorio queda libre. Hombr

In [22]:
constitucion1_agent(query=query,
                    num_chunks=3,
                    stream=False)['user_refine_content']

"\nNow it's your turn. We have provided an existing answer: La Constitución asegura a todas las personas el derecho a la vida y a la integridad física y psíquica [1]. También garantiza la igualdad ante la ley, sin establecer diferencias arbitrarias [1]. Además, establece el deber de respeto a Chile y a sus emblemas nacionales [3].. Below are several numbered sources of information. Use them to refine the existing answer. If the provided sources are not helpful, you will repeat the existing answer.\nBegin refining!\n------\nSource 1:\nCapítulo III: DE LOS DERECHOS Y DEBERES CONSTITUCIONALES\nArtículo 19\n\nLa Constitución asegura a todas las personas:\n1º.- El derecho a la vida y a la integridad física y psíquica de la persona.\nLa ley protege la vida del que está por nacer.\nLa pena de muerte sólo podrá establecerse por delito contemplado en ley aprobada con quórum calificado.\nSe prohíbe la aplicación de todo apremio ilegítimo.\nEl desarrollo científico y tecnológico estará al servici